### Importing the Neccessary Modules

In [9]:
# Importing the Necessary Modules for Wallet Explorer for Realized and Unrealized PnL
import os
import json
import time
import datetime
from datetime import datetime, timedelta
import logging
import threading
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import asyncio
import aiohttp
import streamlit as st

from dataclasses import dataclass
from enum import Enum
from typing import List, Dict, Optional, Tuple

from dotenv import load_dotenv
from dune_client.client import DuneClient


from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

In [10]:
# Load query output from Dune

load_dotenv()
API_KEY = os.getenv('DUNE_CLIENT_API')
coingecko_api_key = os.getenv('GECKO_API_KEY')
dune = DuneClient(API_KEY)
response = dune.get_custom_endpoint_result(
  "firstbml",
  "wallet-explorer-drilldown",
  limit = 100
)

In [11]:
query_id = 5687370  

# 4. Get latest results from Dune
result = dune.get_latest_result(query_id)

# 5. Convert results to pandas DataFrame
df = pd.DataFrame(result.result.rows)

pd.set_option("display.max_columns", None)   # show all columns
pd.set_option("display.width", 2000)         # set a very wide display
pd.set_option("display.colheader_justify", "center")  # nicer alignment
# 6. Display
print(df.head(100))

      action            block_time          blockchain   gas_usd               token_in_address               token_in_amount token_in_symbol              token_out_address               token_out_amount token_out_symbol  trade_value_usd                      tx_hash                                          wallet                  
0   withdrawal  2025-08-03 06:43:19.000 UTC  optimism   0.000836                                        None           NaN          None       0xef4461891dfb3ac8572ccf7c794664a8dd927945      86.912010           WCT           25.615999     0x73e3a1b422468e6068259165ed9a8fee610bab9d8d1f...  0x0b23b218c08dd2156ceb19af5bb765096d73ba70
1   withdrawal  2025-07-17 09:25:41.000 UTC  optimism   0.001434                                        None           NaN          None       0xef4461891dfb3ac8572ccf7c794664a8dd927945      86.912010           WCT           29.120264     0x79d407633c7c8c9bac756660ad0646b2d8bb8aa85468...  0x0b23b218c08dd2156ceb19af5bb765096d73ba70
2

In [12]:
headers = {  
    "accept": "application/json",  # Tells the API server to return the response in JSON format
    "x-cg-demo-api-key":coingecko_api_key  # Sends your CoinGecko API key to authenticate the request (used for Pro API)
}

In [13]:
def build_dune_parameters(parameters: dict):
    """
    Build Dune parameters for API queries.

    Parameters:
    - parameters: dict with keys "wallet", "blockchains", "start_date", "end_date"

    Returns:
    - List of dicts in Dune API parameter format
    """
    dune_parameters = []

    for key, value in parameters.items():
        if key == "blockchains":
            # Format blockchains for SQL IN clause: ('ethereum'),('bnb'),('optimism')
            if isinstance(value, list):
                value = ",".join([f"('{chain}')" for chain in value])
            dune_parameters.append({
                "key": key,
                "type": "text",
                "value": value
            })

        elif key == "wallet":
            # Always wrap wallet address in single quotes
            dune_parameters.append({
                "key": key,
                "type": "text",
                "value": f"'{value}'"
            })

        elif key in ["start_date", "end_date"]:
            # Dune SQL: wrap parameter in quotes only; do NOT include TIMESTAMP here
            # We'll do TIMESTAMP casting in the SQL query itself
            dune_parameters.append({
                "key": key,
                "type": "text",
                "value": f"'{value}'"
            })

        else:
            # Default: cast to string
            dune_parameters.append({
                "key": key,
                "type": "text",
                "value": str(value)
            })

    return dune_parameters

parameters = {
    "wallet": "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "blockchains": ["ethereum", "bnb", "optimism"],
    "start_date": "2025-07-01",
    "end_date": "2025-08-31"
}

dune_parameters = build_dune_parameters(parameters)
print(dune_parameters)



[{'key': 'wallet', 'type': 'text', 'value': "'0x742d35Cc6634C0532925a3b844Bc454e4438f44e'"}, {'key': 'blockchains', 'type': 'text', 'value': "('ethereum'),('bnb'),('optimism')"}, {'key': 'start_date', 'type': 'text', 'value': "'2025-07-01'"}, {'key': 'end_date', 'type': 'text', 'value': "'2025-08-31'"}]


In [14]:
import requests
import json

API_KEY = os.getenv('DUNE_CLIENT_API')
headers = {
    'X-DUNE-API-KEY': API_KEY,
    'Content-Type': 'application/json'
}

# Check both failed executions for error details
execution_ids = ["01K418KY9VJ3Z4R13ZVKCFF112", "01K418Q81TDDFSQVNYT4D1709C"]

for exec_id in execution_ids:
    print(f"\n=== Execution {exec_id} ===")
    
    # Get detailed status
    status_url = f'https://api.dune.com/api/v1/execution/{exec_id}/status'
    response = requests.get(status_url, headers=headers)
    status_data = response.json()
    
    print(f"State: {status_data.get('state')}")
    print(f"Error: {status_data.get('error', 'No error field')}")
    
    # Try to get results anyway (sometimes error details are there)
    results_url = f'https://api.dune.com/api/v1/execution/{exec_id}/results'
    try:
        results_response = requests.get(results_url, headers=headers)
        results_data = results_response.json()
        print(f"Results response: {json.dumps(results_data, indent=2)}")
    except Exception as e:
        print(f"Results fetch error: {e}")


=== Execution 01K418KY9VJ3Z4R13ZVKCFF112 ===
State: QUERY_STATE_FAILED
Error: No error field
Results response: {
  "execution_id": "01K418KY9VJ3Z4R13ZVKCFF112",
  "query_id": 5687370,
  "is_execution_finished": true,
  "state": "QUERY_STATE_FAILED",
  "submitted_at": "2025-08-31T23:50:20.732093Z",
  "expires_at": "2025-11-29T23:50:20.82588Z",
  "execution_started_at": "0001-01-01T00:00:00Z",
  "execution_ended_at": "2025-08-31T23:50:20.825879Z",
  "error": {
    "type": "FAILED_TYPE_EXECUTION_FAILED",
    "message": "line 15:25: mismatched input 'IN'. Expecting: ')', 'AND', 'EXCEPT', 'FETCH', 'GROUP', 'HAVING', 'INTERSECT', 'LIMIT', 'OFFSET', 'OR', 'ORDER', 'UNION', 'WINDOW' at line 15, position 25 [Execution ID: 01K418KY9VJ3Z4R13ZVKCFF112]",
    "metadata": {
      "line": 15,
      "column": 25
    }
  }
}

=== Execution 01K418Q81TDDFSQVNYT4D1709C ===
State: QUERY_STATE_FAILED
Error: No error field
Results response: {
  "execution_id": "01K418Q81TDDFSQVNYT4D1709C",
  "query_id": 568

In [15]:
from dune_client.client import DuneClient
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('DUNE_CLIENT_API')
dune = DuneClient(api_key=API_KEY)

# Test 1: Check if query exists
try:
    query_metadata = dune.get_query(5687370)
    print("✅ Query exists and is accessible")
    print(f"Query name: {query_metadata.name}")
    print(f"Query owner: {query_metadata.owner}")
    print(f"Is private: {query_metadata.is_private}")
except Exception as e:
    print(f"❌ Cannot access query 5687370: {e}")

# Test 2: Try a simple public query to test your API key
try:
    # This is a simple public query that should work
    test_query = QueryBase(query_id=1)  # Usually a basic test query
    test_execution = dune.execute_query(test_query)
    print(f"✅ API key works - test execution: {test_execution.execution_id}")
except Exception as e:
    print(f"❌ API key or basic execution issue: {e}")

# Test 3: Check API key permissions
try:
    # Try to list your queries (if any)
    user_queries = dune.get_queries()  # This might not work with all API keys
    print(f"✅ User queries accessible: {len(user_queries)}")
except Exception as e:
    print(f"❌ Cannot access user queries: {e}")

❌ Cannot access query 5687370: 'query_id'


2025-09-02 14:44:55,107 INFO dune_client.api.base executing 1 on medium cluster
2025-09-02 14:44:55,466 ERROR dune_client.models Can't build ExecutionResponse from {'error': 'Deprecated query engine, please see: https://dune.com/docs/query/Old-Query-Engines/'} due to KeyError: 'execution_id'


❌ API key or basic execution issue: Can't build ExecutionResponse from {'error': 'Deprecated query engine, please see: https://dune.com/docs/query/Old-Query-Engines/'}
❌ Cannot access user queries: 'DuneClient' object has no attribute 'get_queries'


In [16]:
from dune_client.client import DuneClient
import os
from dotenv import load_dotenv
from dune_client.query import QueryBase
from dune_client.types import QueryParameter


# Load API key
load_dotenv()
API_KEY = os.getenv("DUNE_CLIENT_API")

dune = DuneClient(API_KEY)

# Parameters must be simple dicts
parameters = [
    {"key": "wallet", "type": "text", "value": "0x742d35Cc6634C0532925a3b844Bc454e4438f44e"},
    {"key": "blockchains", "type": "text", "value": "ethereum, bnb, optimism"},
    {"key": "start_date", "type": "text", "value": "2025-07-01"},
    {"key": "end_date", "type": "text", "value": "2025-08-31"}
]

query = QueryBase(
    id=5687370,
    name="Wallet Explorer Drilldown",
    description="Drilldown for Wallet Explorer",
    parameters=[QueryParameter(**param) for param in parameters]
)
# Run query and fetch results
results = dune.get_query_results(query_id=query.id, parameters=parameters)

print("Query results:")
print(results)


TypeError: QueryParameter.__init__() got an unexpected keyword argument 'key'

In [ ]:
from dune_client.client import DuneClient
from dune_client.types import QueryParameter, DuneQuery
import os

# Load your API key
DUNE_API_KEY = os.getenv("DUNE_API_KEY")  # make sure it's set in your env
client = DuneClient(api_key=DUNE_API_KEY)

# Define query
query = DuneQuery(
    query_id=5687370,   # Replace with your actual query ID
    params=[
        # Add params only if your query has variables
        # QueryParameter.text(name="wallet_address", value="0x123..."),
        # QueryParameter.number(name="limit", value=100),
    ]
)

# Run a fresh execution (not cached)
execution = client.refresh(query)

# Wait until execution finishes & fetch results
results = client.get_execution_results(execution.execution_id)

print(results)


ImportError: cannot import name 'DuneQuery' from 'dune_client.types' (c:\Users\g\Documents\PnL Wallet Explorer\WalletPnL\Lib\site-packages\dune_client\types.py)

In [ ]:
import dune_client.types as t
print(dir(t))


['Address', 'Any', 'Dict', 'DuneRecord', 'Enum', 'ParameterType', 'QueryParameter', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'annotations', 'datetime', 'postgres_date', 're']


In [ ]:
from dune_client.client import DuneClient
from dune_client.query import QueryBase
from dune_client.types import QueryParameter
import os

# Load your API key from environment variable
DUNE_API_KEY = os.getenv("DUNE_CLIENT_API")  # make sure you've set this
client = DuneClient(api_key=DUNE_API_KEY)

# Define your query (replace with your actual query ID)
query = QueryBase(
    query_id=5687370,
    params=[
        # Add parameters only if your query expects them
        # Example:
        # QueryParameter.text_type("wallet_address", "0x123..."),
        # QueryParameter.number_type("limit", 100),
    ]
)

# Run a fresh execution (not cached)
execution = client.refresh(query)

# Fetch results by execution ID
results = client.get_execution_results(execution.execution_id)

# Print results
print(results)


C:\Users\g\AppData\Local\Temp\ipykernel_24404\54702744.py:22: DeprecationWarning: Call to deprecated method refresh. (Please use run_query) -- Deprecated since version 1.2.1.
  execution = client.refresh(query)
2025-09-01 17:56:42,475 INFO dune_client.api.base executing 5687370 on medium cluster
2025-09-01 17:56:44,652 INFO dune_client.api.base waiting for query execution 01K433BBKRMP8071G3FAM1HVSY to complete: ExecutionState.PENDING (queue position: 631)
2025-09-01 17:56:45,850 INFO dune_client.api.base waiting for query execution 01K433BBKRMP8071G3FAM1HVSY to complete: ExecutionState.PENDING (queue position: 629)
2025-09-01 17:56:47,068 ERROR dune_client.api.base ExecutionState.FAILED: execution_id=01K433BBKRMP8071G3FAM1HVSY, query_id=5687370, times=TimeData(submitted_at=datetime.datetime(2025, 9, 1, 16, 56, 45, 433212, tzinfo=tzutc()), execution_started_at=None, execution_ended_at=None, expires_at=datetime.datetime(2025, 11, 30, 16, 56, 46, 899733, tzinfo=tzutc()), cancelled_at=None

QueryFailed: Error data: None

In [ ]:
import os
import pandas as pd
import streamlit as st
from dune_client.client import DuneClient
from dune_client.query import QueryBase
from dune_client.types import QueryParameter
from dotenv import load_dotenv

# Load API key
load_dotenv()
API_KEY = os.getenv("DUNE_CLIENT_API")
dune = DuneClient(API_KEY)

st.set_page_config(page_title="Wallet PnL Explorer", layout="wide")

st.title("💰 Wallet PnL Explorer")

# --- User Inputs ---
wallet = st.text_input("Enter Wallet Address", "0x1234567890abcdef...")

blockchains = st.multiselect(
    "Select Blockchains",
    ["ethereum", "optimism", "arbitrum", "bnb", "base"],
    default=["ethereum"]
)

start_date = st.date_input("Start Date")
end_date = st.date_input("End Date")

if st.button("Run Analysis"):
    with st.spinner("Fetching fresh results from Dune... ⛏️"):

        # Define query with parameters
        query = QueryBase(
            query_id=5687370,
            params=[
                QueryParameter.array_type("blockchains", blockchains),
                QueryParameter.text_type("wallet", wallet),
                QueryParameter.text_type("start_date", str(start_date) + " 00:00:00"),
                QueryParameter.text_type("end_date", str(end_date) + " 23:59:59")
            ]
        )

        # Run query fresh
        execution = dune.run_query(query)
        result = dune.get_execution_results(execution.execution_id)

        # Convert to DataFrame
        df = pd.DataFrame(result.result.rows)

        if df.empty:
            st.warning("No results found for this wallet and date range.")
        else:
            st.success(f"Fetched {len(df)} transactions ✅")
            st.dataframe(df)

            # Basic PnL calculation example
            df["net_usd"] = df["trade_value_usd"].astype(float) - df["gas_usd"].astype(float)
            pnl = df["net_usd"].sum()

            st.metric("Net PnL (USD)", f"${pnl:,.2f}")


2025-09-02 08:15:21.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.484 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-02 08:15:21.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar